In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale,Normalizer,OneHotEncoder
from sklearn.preprocessing import normalize
from keras.models import Sequential,load_model
from tensorflow.python.keras.saving.saved_model import load as saved_model_load
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
import keras.optimizers 
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('task42.training.csv')

In [3]:
train2 = pd.read_csv('task42.training.csv')

In [4]:
test = pd.read_csv('task42.test.WITHOUT.answers.csv')

In [5]:
train_answer = train['class']

In [6]:
train['class']

0        I
1        F
2        B
3        G
4        J
        ..
29995    C
29996    C
29997    F
29998    D
29999    D
Name: class, Length: 30000, dtype: object

In [7]:
import matplotlib.pyplot as plt
plt.hist(train['f24'])
plt.show()

<Figure size 640x480 with 1 Axes>

In [8]:
train_last = pd.get_dummies(train['f24'])
test_last = pd.get_dummies(test['f24'])

In [9]:
train_last

,A,B,C,D,E,F,G,H,I,J
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
29995,0,0,0,0,0,1,0,0,0,0
29996,0,0,0,0,0,0,0,0,0,1
29997,0,0,0,0,0,0,0,1,0,0
29998,0,1,0,0,0,0,0,0,0,0


In [10]:
from sklearn.decomposition import FastICA 
ICA = FastICA(n_components=1, random_state=12) 
train_last=ICA.fit_transform(train_last)
test_last = ICA.fit_transform(test_last)

In [11]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=4)
# train_last=pca.fit_transform(train_last)
# test_last = pca.fit_transform(test_last)

In [12]:
train_last

array([[-0.00524062],
       [-0.00119252],
       [-0.00119252],
       ...,
       [-0.00097408],
       [-0.00132454],
       [-0.00097408]])

In [13]:
train = train.drop(columns = 'class')
train = train.drop(columns = 'f24')
test = test.drop(columns = 'f24')

In [14]:
train = np.array(train,dtype=np.float32)
test = np.array(test,dtype=np.float32)

In [15]:
meanlist=[]
stdlist =[]

In [16]:
sc = StandardScaler()
sc.fit(train)
train = sc.transform(train)
test = sc.transform(test)

In [17]:
# for i in range(np.shape(train)[1]):
#     tmp = np.sum(train[:,i])
#     mean = tmp/np.shape(train)[0]
#     meanlist.append(mean)
#     std = np.std(train[:,i])
#     stdlist.append(std)
#     for j in range(np.shape(train)[0]):
#         train[j][i] = (train[j][i]-mean)/std

In [18]:
# meanlist

In [19]:
# stdlist

In [20]:
# for i in range(np.shape(test)[1]):
#     mean = meanlist[i]
#     std  = stdlist[i]
#     for j in range(np.shape(test)[0]):
#         test[j][i] = (test[j][i]-mean)/std

In [21]:
# normalizer = Normalizer().fit(train) 
# train = normalizer.transform(train)

In [22]:
train = np.append(train,train_last,axis=1)
test = np.append(test,test_last,axis=1)

In [23]:
np.shape(train)

(30000, 25)

In [24]:
ans = pd.get_dummies(train_answer)

In [25]:
ans = np.array(ans)

In [26]:
train_feature_len = np.shape(train)[1]

In [27]:
train_x,val_x,train_y,val_y = train_test_split(train,ans,test_size=0.3,random_state=44)

## Decision Tree (0.10100)

In [30]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(train_x, train_y)

In [31]:
decisiontree_ans =  clf.predict(val_x)

In [32]:
f1_score(decisiontree_ans,val_y,average='weighted')

0.36290759509239306

In [33]:
# alph = ['A','B','C','D','E','F','G','H','I','J']
# prediction_final=[]
# for i in range(np.shape(decisiontree_ans)[0]):
#     index = np.argmax(decisiontree_ans[i], axis=0)
#     prediction_final.append(alph[index])

In [34]:
# tt = dict([('id',np.array(range(1,2001))),('Predicted',prediction_final)])
# tt = pd.DataFrame(tt)
# tt.to_csv('test_decision_tree.csv',index=False)

## KNN (0.01919)

In [35]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(train_x, train_y)
knn_ans = knn.predict(val_x)

In [36]:
f1_score(knn_ans,val_y,average='weighted')

0.3728689519133481

## SVM(0.1044)

In [37]:
train_x2,val_x2,train_y2,val_y2 = train_test_split(train,train2['class'].values,test_size=0.3,random_state=44)

In [39]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_x2, train_y2 )
svm_ans = clf.predict(val_x)

In [40]:
f1_score(svm_ans,val_y2,average='weighted')

0.8459108084998486

In [28]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [29]:
lr= 1e-3
opti =keras.optimizers.Adam(learning_rate=lr)
model3 = Sequential()  
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*2, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(train_feature_len*1, activation = 'relu',use_bias=True))
model3.add(BatchNormalization())
model3.add(Dense(10, activation = 'softmax',use_bias=True))
para3 = model3.compile(optimizer = opti, loss = 'categorical_crossentropy', metrics = ['acc',f1_m,precision_m, recall_m])


In [30]:
# model3=load_model("weights_with_normalize.best.hdf5",custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})

In [31]:
# checkpoint
filepath="weights_with_normalize.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
mode='max')
callbacks_list = [checkpoint]

In [32]:
#model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [33]:
# print('Training ------------')
# model3.fit(train_x, train_y, validation_split=0.33, epochs=1500,batch_size=128,callbacks=callbacks_list,verbose=0) #2000

In [250]:
print('Training ------------')
model3.fit(train_x, train_y, epochs=5,batch_size=256) #2000

Training ------------
Epoch 1/5
83/83 [==============================] - 0s 3ms/step - loss: 0.0455 - acc: 0.9838 - f1_m: 0.9840 - precision_m: 0.9840 - recall_m: 0.9840
Epoch 2/5
83/83 [==============================] - 0s 3ms/step - loss: 0.0427 - acc: 0.9845 - f1_m: 0.9818 - precision_m: 0.9818 - recall_m: 0.9818
Epoch 3/5
83/83 [==============================] - 0s 2ms/step - loss: 0.0468 - acc: 0.9818 - f1_m: 0.9806 - precision_m: 0.9806 - recall_m: 0.9806
Epoch 4/5
83/83 [==============================] - 0s 3ms/step - loss: 0.0544 - acc: 0.9787 - f1_m: 0.9760 - precision_m: 0.9760 - recall_m: 0.9760
Epoch 5/5
83/83 [==============================] - 0s 2ms/step - loss: 0.0626 - acc: 0.9758 - f1_m: 0.9746 - precision_m: 0.9746 - recall_m: 0.9746


In [251]:
# model3.save('ML_task2.h5')

In [252]:
# model3 = load_model('weights.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
# model3 = load_model('weights_with_normalize.best.hdf5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})


In [253]:
print('\nTesting ------------')
loss, accuracy, f1_score, precision, recall = model3.evaluate(val_x, val_y, verbose=0)
print('\ntest loss: ', loss)
print('\ntest f1_score: ', f1_score) 
#pca epoch400 0.971907377243042
#都沒做 0.91
#fastica 0.95...


Testing ------------

test loss:  0.08593280613422394

test f1_score:  0.961768627166748


In [254]:
prediction = model3.predict(test)

In [242]:
np.argmax(prediction[0], axis=0)

4

In [243]:
alph = ['A','B','C','D','E','F','G','H','I','J']

In [244]:
prediction_final=[]

In [245]:
for i in range(np.shape(prediction)[0]):
    index = np.argmax(prediction[i], axis=0)
    prediction_final.append(alph[index])

In [246]:
# prediction_final

In [247]:
tt = dict([('id',np.array(range(1,6001))),('Predicted',prediction_final)])

In [248]:
tt = pd.DataFrame(tt)

In [249]:
tt.to_csv('test42.csv',index=False)